# Un primer modelo generativo de texto

En este notebook, usaremos los conceptos de probabilidad (condicional) y n-gramas para generar texto (similar a como lo hace ChatGPT o Gemini).

## Carga de un libro

## Histogramas de palabras y letras

## Generación vía letras

## Generación vía palabras

Supongamos que tenemos un vocabulario con $k$ palabras distintas y $n$ es la longitud de nuestro texto.  

Si modelamos que cada palabra en el texto es elegida de forma independiente según una distribución de probabilidades $(p_1, p_2, \dots, p_k)$ sobre el vocabulario, entonces el vector de conteos $(X_1, X_2, \dots, X_k)$, donde $X_i$ es el número de veces que aparece la palabra $i$, sigue una distribución multinomial:  

$$(X_1, X_2, \dots, X_k) \sim \text{Multinomial}(n, (p_1, p_2, \dots, p_k))$$  

Este modelo es una base común en procesamiento de lenguaje natural, aunque en la práctica las palabras no son independientes.


In [ ]:
vocabulario = [
    'el', 'la', 'de', 'en', 'un',     # palabras funcionales
    'gato', 'niña', 'perro', 'casa', 'alfombra',  # sustantivos
    'duerme', 'juega', 'come'  # verbos
]

probabilidades = [
    0.12, 0.10, 0.10, 0.06, 0.06,  # funcionales: 0.44
    0.10, 0.08, 0.08, 0.08, 0.06,  # sustantivos: 0.40
    0.06, 0.05, 0.05  # verbos: 0.15
]

# Número de palabras en el fragmento de texto
n_palabras = 30

# Simulamos el fragmento de texto como una secuencia de palabras
fragmento = np.random.choice(vocabulario, size=n_palabras, p=probabilidades)

# Contamos la frecuencia de cada palabra
unique, counts = np.unique(fragmento, return_counts=True)

# Visualización
plt.bar(unique, counts, color='skyblue')
plt.title(f'Conteo multinomial de palabras en un fragmento de {n_palabras} palabras')
plt.xlabel('Palabras')
plt.ylabel('Frecuencia')
plt.show()

print("Fragmento generado:")
print(' '.join(fragmento))